In [1]:
#Import of Python available libraries
import sculib

import datetime
from astropy.time import Time

#mpi = sculib.scu()
#mpi.ip = '10.96.64.10'
sim = sculib.scu()
sim.ip = '192.168.0.106'

In [2]:
def gen_ramp(az0, az1, el0, el1, start, duration):
    '''
    start and end azimuth,
    start and end elevation
    start time (datetime) and duration in seconds
    '''
    #t = [Time( start +  datetime.timedelta(seconds = t) ).mjd for t in raw_t]
    t = [ start +  datetime.timedelta(seconds = t) for t in range(0, duration + 1) ]
    az = [az0 + x * (az1-az0)/duration for x in range(len(t))]
    el = [el0 + x * (el1-el0)/duration for x in range(len(t))]
    return({'t': Time(t).mjd, 'az': az, 'el': el})

def ramp_table(azimuth, elevation):
    angle_duration_pair = [ 
    [0.0001, 120],
    [0.001, 120],
    [0.01, 120],
    [0.1, 120],
    [0.3, 120],
    [0.5, 120],
    [1, 60] ]
    #this portion generates the track table for all the sweeps at a particular azimuth/elevation
    sign = 1
    body = '' #body is the track table string, starts empty
    offset = 30 #time offset to start of track table
    t_prep = 30 #time preparation between sweeps
    old_aoff = 0 #need to keep track of end position of previous sweep
    t =  datetime.datetime.utcnow() + datetime.timedelta(seconds = offset) 
    for angle_off, duration in angle_duration_pair:
        print("\t Angle offset: {} Duration: {}".format(angle_off, 2*duration))       
        #this generates track table segment from end of previous sweep to start of next sweep
        body += sim.format_body(**gen_ramp(azimuth-sign*old_aoff, azimuth-sign*angle_off, elevation, elevation, t, t_prep))
        t = t + datetime.timedelta(seconds = t_prep + 1)
        #this generates track table segment for the sweep itself
        body += sim.format_body(**gen_ramp(azimuth-sign*angle_off, azimuth+sign*angle_off, elevation, elevation, t, 2*duration))
        t = t + datetime.timedelta(seconds = 2*duration + 1)
        #we change sign every time to sweep from opposite ends, for efficiency
        sign = -sign
        #we remember the previous end of the sweep to prepare for next sweep
        old_aoff = angle_off
    return(body)

def mpi_t682(scu):
    print('Test 6.8.2')

    #Revisions:
    #0.1 initial write - could do with some kat sensor retrieval
    #0.2 rewrite for sculib

    #Common setup
    config_name = 'HN_TILT_TEST'

    
    azimuths = [-180, -120, -60, 0, 60, 120 ,180]
    az_start = -180
    az_speed = 3
    
    elevations = [15, 52, 85]
    el_start = 15 # 
    el_speed  = 1 #  slew rate of 1 should be possible after retrofit


    
    wait1 =1 
    wait5 = 5
    wait10 = 10
    wait100 = 100
    sampling = 200
    scu.debug = False
    #debug test only remove these
    '''
    elevations = [15]
    azimuths = [-180]
    wait5 = 1
    wait10 = 1
    wait100 = 1
    sampling = 5000
    scu.debug = False
    '''
  
    '''
    Track table with position coordinates at 1 second intervals to represent a ramp with the following rates and durations:
    +0.0001°/sec for 120 sec, -0.0001°/sec for 120 sec
    +0.001°/sec for 120 sec, -0.001°/sec for 120 sec
    +0.01°/sec for 120 sec, -0.01°/sec for 120 sec
    +0.1°/sec for 120 sec, -0.1°/sec for 120 sec
    +0.3°/sec for 120 sec, -0.3°/sec for 120 sec
    +0.5°/sec for 120 sec, -0.5°/sec for 120 sec
    +1°/sec for 60 sec, -1°/sec for 60 sec

    1) Manually move the Dish to 15° Elevation, -180° Azimuth and de-activate all axes
    2) Activate the Azimuth and Elevation axis
    • wait for 15 seconds to complete the activation process
    3) Command Elevation to 15° at 1°/sec and Azimuth to -180° at 3°/sec and (“Move to Absolute Position”)
    • wait for 70 seconds to complete the move and to stabilise
    4) Upload the Track Table with test profile defined below to the SCU
    • wait for 5 seconds
    5) Start the ACU data recording on the SCU using logging configuration TBD
    • wait for 5 seconds
    6) Command Azimuth to start tracking the loaded table
    • wait for 1080 seconds to complete the track table
    7) Stop the ACU data recording
    8) Repeat steps 3) to 7) for Elevation angles of 52° and 85°
    9) De-activate all axis
    10) Repeat steps 2) to 9) with Azimuth angles of 120°, 60°, 0°, 60°, 120° and 180°
    11) Extract the data on the SCU @ 5Hz sampling
    12) Rename the files to TBD (eg. 6p8p2_i.csv)
    13) Download the MeerKAT weather station data for the test period
    '''

    #Step 1
    #Manual move dish to  15° Elevation, -180° Azimuth
    scu.activate_dmc()
    scu.wait_duration(wait10*2) 
    
    print("Manual move to az: {} and el: {}".format(az_start, el_start))
    az_angle = az_start
    scu.abs_azimuth(az_angle, az_speed)
    scu.wait_duration(wait10*(12-7))

    el_angle = el_start
    scu.abs_elevation(el_angle, el_speed)
    scu.wait_duration(wait10*7)     #scu.reset_dmc()

    #Step 2 lets just skip an activate and deactive

    #Step 3 - command 
  
    #Step 4 - Load track table
    for azimuth in azimuths:
        print("\t Az center: {} ".format(azimuth)) 
        az_angle = azimuth
        scu.abs_azimuth(az_angle, az_speed)
        scu.wait_duration(wait10*(12-7))
        for elevation in elevations:
            el_angle = elevation
            scu.abs_elevation(el_angle, el_speed)
            scu.wait_duration(wait10*4) 
            print("\t El center: {} ".format(elevation))
            #Step 5 - start ACU recording on SCU
            if scu.logger_state() == 'RECORDING':
                print('WARNING, already recording - attempting to stop and start a fresh logger')
                scu.stop_logger()  
                scu.wait_duration(wait10)
            if scu.logger_state() == 'STOPPED':
                print('Starting logger with config: {}'.format(config_name))
                scu.start_logger(config_name)
                scu.wait_duration(wait10)

            #fire and forget the track table generated for this azimuth and elevation
            scu.acu_ska_track(ramp_table(azimuth, elevation))
            scu.wait_duration(wait10*3)  #time offset is 30s as defined by offset abovet
                
            #Step 6 - Command Azimuth to start tracking the loaded table
            scu.wait_duration(wait10*6*(26+4))  #It takes 30 minutes for all the sweeps             
                
            #Step 7) Stop the ACU data recording
            scu.stop_logger()
            scu.wait_duration(wait10)
            print(scu.logger_state())

            #Step 11) Extract the data on the SCU @ 5Hz sampling
            #Step 12) Rename the files to TBD (eg. 6p8p2_i.csv)
            print("Rename the file to TBD (eg. 6p8p2_i.csv)")
            scu.save_session('6p8p2_i_az_' + str(azimuth) + '_el_' + str(elevation), interval_ms = sampling)

        #Step 8) Repeat steps 3) to 7) for Elevation angles of 52° and 85°
        #Step 9) De-activate all axis ? IGNORED here done at end.
        #Step 10) Repeat steps 2) to 9) with Azimuth angles of 120°, 60°, 0°, 60°, 120° and 180°
        
        #13) Download the MeerKAT weather station data for the test period

    #Step x) De-activate all axis
    scu.deactivate_dmc()
    scu.wait_duration(wait10*2) 
    

In [3]:
#mpi_t682(mpi)
mpi_t682(sim)


Test 6.8.2
activate dmc
wait for 20.0s done *
Manual move to az: -180 and el: 15
abs az: -180.0000 vel: 3.0000
wait for 50.0s done *
abs el: 15.0000 vel: 1.0000
wait for 70.0s done *
	 Az center: -180 
abs az: -180.0000 vel: 3.0000
wait for 50.0s done *
abs el: 15.0000 vel: 1.0000
wait for 40.0s done *
	 El center: 15 
logger state 
WARNING, already recording - attempting to stop and start a fresh logger
stop logger
wait for 10.0s done *
logger state 
Starting logger with config: HN_TILT_TEST
start logger:  HN_TILT_TEST
wait for 10.0s done *
	 Angle offset: 0.0001 Duration: 240
	 Angle offset: 0.001 Duration: 240
	 Angle offset: 0.01 Duration: 240
	 Angle offset: 0.1 Duration: 240
	 Angle offset: 0.3 Duration: 240
	 Angle offset: 0.5 Duration: 240
	 Angle offset: 1 Duration: 120
acu ska track
wait for 30.0s done *
wait for 1800.0s done *
stop logger
wait for 10.0s done *
logger state 
STOPPED
Rename the file to TBD (eg. 6p8p2_i.csv)
Attempt export and save of session: last at rate 200 

KeyboardInterrupt: 